# Here the code will give the flatten duck curve without user's input of storage.

In [35]:
import pandas as pd

# ─────────────────────────────────────────────────────────────
# 1. INPUTS
# ─────────────────────────────────────────────────────────────
installed_capacity = {
    2020: 36050.74,
    2021: 46275.14,
    2022: 60813.93,
    2023: 71780.74,
    2024: 90762.10,
    2030: 280000.00
}

poa_csv     = r"D:\IIT BOMBAY\May 2025\Duck_Curve\POA.csv"
demand_xlsx = r"D:\IIT BOMBAY\May 2025\Duck_Curve\Demand 2020_24.xlsx"
output_xlsx = r"D:\IIT BOMBAY\May 2025\Duck_Curve\combined_data_with_storage3.xlsx"

# ─────────────────────────────────────────────────────────────
# 2. READ  POA  +  DEMAND   →  BASIC NET-LOAD
# ─────────────────────────────────────────────────────────────
df = pd.read_csv(poa_csv)
df['datetime']  = pd.to_datetime(df['timestamp'])
df['year']      = df['datetime'].dt.year
df['month_day'] = df['datetime'].dt.strftime('%m-%d')
df['hour']      = df['datetime'].dt.hour

# Plane-of-array → MW
df['solar_generation'] = df.apply(
    lambda r: installed_capacity[r.year] * r.poa / 1000, axis=1
)

demand = pd.read_excel(demand_xlsx)
if len(df) != len(demand):
    raise ValueError("POA rows ≠ demand rows")
df['demand']   = demand['Hourly Demand Met (in MW)']
df['net_load'] = df['demand'] - df['solar_generation']

# ─────────────────────────────────────────────────────────────
# 3. 2030-ONLY ▸ DAYTIME SURPLUS & ANYTIME DISCHARGE
# ─────────────────────────────────────────────────────────────
df['solar_stored_day']          = 0.0   # MWh h⁻¹
df['storage_discharged_night']  = 0.0

mask2030 = df['year'] == 2030
for day in df.loc[mask2030, 'month_day'].unique():
    sel = mask2030 & (df['month_day'] == day)
    avg_nl = df.loc[sel, 'net_load'].mean()  # daily reference level

    # DAY: 07–18 → charge any positive surplus
    day_hours = sel & df['hour'].between(7, 18)
    df.loc[day_hours, 'solar_stored_day'] = (
        (avg_nl - df.loc[day_hours, 'net_load']).clip(lower=0)
    )

    # ANYTIME → discharge whenever net_load > avg_nl
    df.loc[sel, 'storage_discharged_night'] = (
        (df.loc[sel, 'net_load'] - avg_nl).clip(lower=0)
    )

# ─────────────────────────────────────────────────────────────
# 4. FINAL DUCK CURVE (2030 WITH STORAGE)
# ─────────────────────────────────────────────────────────────
df['net_load_final'] = (
      df['demand']
    - df['solar_generation']
    + df['solar_stored_day']
    - df['storage_discharged_night']
)

df.to_excel(output_xlsx, index=False)
print("Saved →", output_xlsx)


C:\Users\sumit\AppData\Local\Temp\ipykernel_17708\314015804.py:23: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['datetime']  = pd.to_datetime(df['timestamp'])


Saved → D:\IIT BOMBAY\May 2025\Duck_Curve\combined_data_with_storage3.xlsx


In [37]:
import pandas as pd
import matplotlib.pyplot as plt
import os

df = pd.read_excel(r"D:\IIT BOMBAY\May 2025\Duck_Curve\combined_data_with_storage3.xlsx")

# rebuild helper columns if necessary
if 'hour_shifted' not in df.columns:
    df['datetime']     = pd.to_datetime(df['timestamp'])
    df['hour']         = df['datetime'].dt.hour
    df['hour_shifted'] = (df['hour'] - 23) % 24    # 23:00 → 0, 22:00 → 1, ...

out_dir = r"D:\IIT BOMBAY\May 2025\Duck_Curve\dc_Oct_upto_2030"
os.makedirs(out_dir, exist_ok=True)

for day in sorted(df['month_day'].unique()):
    today = df[df['month_day'] == day]

    plt.figure(figsize=(10, 5))

    # 2020-2024 + original 2030
    for yr in sorted(today['year'].unique()):
        trace = today[today['year'] == yr].sort_values('hour_shifted')

        # close the 24-h loop
        first = trace.iloc[0].copy(); first['hour_shifted'] = 24
        trace = pd.concat([trace, pd.DataFrame([first])], ignore_index=True)

        ls = '--' if yr == 2030 else '-'
        plt.plot(trace['hour_shifted'], trace['net_load'],
                 linestyle=ls, marker='o', label=f"{yr} Duck")

    # 2030 WITH STORAGE
    new = today[today['year'] == 2030].sort_values('hour_shifted')
    if not new.empty:
        first = new.iloc[0].copy(); first['hour_shifted'] = 24
        new   = pd.concat([new, pd.DataFrame([first])], ignore_index=True)

        plt.plot(new['hour_shifted'], new['net_load_final'],
                 linewidth=2.5, label="2030 Duck Curve with Storage")

    # cosmetics
    xt = list(range(0, 25, 2))
    xl = ["23:00" if x in (0, 24) else f"{(x-1)%24:02d}:00" for x in xt]
    plt.xticks(xt, xl)
    plt.xlabel("Hour of the Day")
    plt.ylabel("Net Load (MW)")
    plt.title(f"Duck Curves vs. 2030-with-Storage — {day}")
    plt.grid(True, ls="--", alpha=0.6)
    plt.legend(title="Year ", fontsize=8)

    fn = os.path.join(out_dir, f"duck_curve_{day}.png")
    plt.savefig(fn, bbox_inches='tight')
    plt.close()

print("Plots saved to", out_dir)

C:\Users\sumit\AppData\Local\Temp\ipykernel_17708\4001896050.py:9: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['datetime']     = pd.to_datetime(df['timestamp'])


Plots saved to D:\IIT BOMBAY\May 2025\Duck_Curve\dc_Oct_upto_2030
